# **The fourth in-class-exercise (40 points in total, 11/03/2022)**

Question description: Please use the text corpus you collected in your last in-class-exercise for this exercise. Perform the following tasks:

In [1]:
import nltk
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import itertools
import lxml.html as LH
import urllib.request as urllib2
from bs4 import BeautifulSoup


nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
def getAllDoxyDonkeyPosts(url,links):
    request = urllib2.Request(url)
    response = urllib2.urlopen(request)
    soup = BeautifulSoup(response)
    for a in soup.findAll('a'):
        try:
            url = a['href']
            title = a['title']
            if title == "Older Posts":
                print(title, url)
                links.append(url)
                getAllDoxyDonkeyPosts(url,links)
        except:
            title = ""
    return


blogUrl = "https://doxydonkey.blogspot.com/"
links = []
getAllDoxyDonkeyPosts(blogUrl,links)

def getDoxyDonkeyText(testUrl):
    request = urllib2.Request(testUrl)
    response = urllib2.urlopen(request)
    soup = BeautifulSoup(response)
    mydivs = soup.findAll("div", {"class":'post-body'})
    

    posts =[]
    for div in mydivs:
        posts+=map(lambda p:str(str(p.text).encode('ascii', errors='replace')).replace("?"," "), div.findAll("li"))
    return posts

doxyDonkeyPosts = []
for link in links:
    doxyDonkeyPosts+=getDoxyDonkeyText(link)

Older Posts https://doxydonkey.blogspot.com/search?updated-max=2017-05-23T19:53:00-07:00&max-results=7
Older Posts https://doxydonkey.blogspot.com/search?updated-max=2017-05-14T19:02:00-07:00&max-results=7&start=7&by-date=false
Older Posts https://doxydonkey.blogspot.com/search?updated-max=2017-05-02T19:43:00-07:00&max-results=7&start=14&by-date=false
Older Posts https://doxydonkey.blogspot.com/search?updated-max=2017-04-17T19:26:00-07:00&max-results=7&start=21&by-date=false
Older Posts https://doxydonkey.blogspot.com/search?updated-max=2017-04-10T18:56:00-07:00&max-results=7&start=28&by-date=false
Older Posts https://doxydonkey.blogspot.com/search?updated-max=2017-03-30T19:57:00-07:00&max-results=7&start=35&by-date=false
Older Posts https://doxydonkey.blogspot.com/search?updated-max=2017-03-20T19:47:00-07:00&max-results=7&start=42&by-date=false
Older Posts https://doxydonkey.blogspot.com/search?updated-max=2017-03-02T17:42:00-08:00&max-results=7&start=49&by-date=false
Older Posts http

## (1) (10 points) Generate K topics by using LDA, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here: 

https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

In [4]:
!pip install pyLDAvis
!pip install --upgrade gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Importing Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Importing all plotting tools
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Importing spacy for lemmatization
import spacy

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [6]:

#Creating Bigram and Trigram Models
data_words = doxyDonkeyPosts
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) 
bigram_mod = gensim.models.phrases.Phraser(bigram)


trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [7]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def removeAllStopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def makeBigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def makeTrigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [8]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')


import spacy.cli
spacy.cli.download("en")


stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

# Remove Stop Words
data_words_nostops = removeAllStopwords(data_words)

# Form Bigrams
data_words_bigrams = makeBigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized)

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


/usr/local/lib/python3.7/dist-packages/spacy/language.py:1899: UserWarning: [W123] Argument disable with value ['parser', 'ner'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  config_value=config["nlp"][key],
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [9]:
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 2), (2, 1), (3, 1), (4, 2), (5, 1), (6, 2), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 2), (19, 2), (20, 1), (21, 1), (22, 1), (23, 3), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 3), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 2), (45, 1), (46, 1), (47, 1), (48, 1), (49, 2), (50, 1), (51, 1), (52, 5), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 3), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 5), (81, 1), (82, 4), (83, 1), (84, 1), (85, 2), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 3), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1)]]


In [10]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('absorb', 1),
  ('accord', 2),
  ('ago', 1),
  ('alarm', 1),
  ('already', 2),
  ('approach', 1),
  ('ask', 2),
  ('atmosphere', 1),
  ('away', 1),
  ('begin', 1),
  ('behemoth', 1),
  ('big', 1),
  ('boost', 1),
  ('cash', 1),
  ('catch', 1),
  ('check', 1),
  ('close', 1),
  ('commitment', 1),
  ('company', 2),
  ('concern', 2),
  ('control', 1),
  ('cool', 1),
  ('day', 1),
  ('deal', 3),
  ('deeply', 1),
  ('desire', 1),
  ('digital', 1),
  ('dominant', 1),
  ('entrepreneur', 1),
  ('feel', 1),
  ('firm', 1),
  ('founder', 1),
  ('fret', 1),
  ('fund', 3),
  ('funding', 1),
  ('give', 1),
  ('hand', 1),
  ('head', 1),
  ('heat', 1),
  ('ignited', 1),
  ('industry', 1),
  ('initial', 1),
  ('initially', 1),
  ('invest', 1),
  ('investor', 2),
  ('japanese', 1),
  ('ladle', 1),
  ('large', 1),
  ('last', 1),
  ('less', 2),
  ('make', 1),
  ('market', 1),
  ('money', 5),
  ('month', 1),
  ('much', 1),
  ('name', 1),
  ('need', 1),
  ('negotiate', 1),
  ('originally', 1),
  ('outlan

In [11]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [12]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.052*"life" + 0.043*"school" + 0.035*"expensive" + 0.033*"man" + '
  '0.032*"battery" + 0.032*"education" + 0.030*"powerful" + 0.022*"accelerate" '
  '+ 0.022*"mile" + 0.020*"consume"'),
 (1,
  '0.285*"apple" + 0.068*"iphone" + 0.065*"phone" + 0.042*"smartphone" + '
  '0.030*"holiday" + 0.030*"sale" + 0.027*"device" + 0.022*"store" + '
  '0.022*"sell" + 0.020*"shipment"'),
 (2,
  '0.213*"uber" + 0.110*"driver" + 0.089*"ride" + 0.075*"traffic" + '
  '0.044*"hail" + 0.042*"tax" + 0.040*"lyft" + 0.030*"car" + 0.017*"city" + '
  '0.012*"transportation"'),
 (3,
  '0.333*"video" + 0.112*"youtube" + 0.053*"view" + 0.031*"extend" + '
  '0.027*"creator" + 0.023*"advertisement" + 0.021*"star" + 0.019*"gaming" + '
  '0.019*"film" + 0.006*"suppose"'),
 (4,
  '0.076*"percent" + 0.049*"revenue" + 0.041*"year" + 0.039*"share" + '
  '0.035*"quarter" + 0.025*"company" + 0.022*"sale" + 0.020*"growth" + '
  '0.020*"profit" + 0.016*"report"'),
 (5,
  '0.067*"voice" + 0.057*"music" + 0.051*"chip" 

## (2) (10 points) Generate K topics by using LSA, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python

In [41]:

#Import modules
import os.path
from gensim import corpora
from gensim.models import LsiModel
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt

In [42]:
# Write your code here
lsamodel = LsiModel(corpus, 20, id2word)

In [43]:
def compute_coherence_values(dictionary, doc_term_matrix, doc_clean, stop, start=2, step=3):
    coherence_values = []
    model_list = []
    for num_topics in range(start, stop, step):
        # generate LSA model
        model = LsiModel(doc_term_matrix, num_topics=20, id2word = dictionary)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=doc_clean, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
    return model_list, coherence_values

In [44]:
start =2
stop = 50
step= 5
model_list_lsa, coherence_values_lsa = compute_coherence_values(id2word, corpus,data_lemmatized,stop, start, step)

IndexError: ignored

In [17]:
for no_of_topics, cv in zip(range(2, 60, 6), coherence_values_lsa):
  print("Num Topics:", no_of_topics, " - Coherence Value:", round(cv, 4))

NameError: ignored

## (3) (10 points) Generate K topics by using  lda2vec, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://nbviewer.org/github/cemoody/lda2vec/blob/master/examples/twenty_newsgroups/lda2vec/lda2vec.ipynb

In [23]:
nltk.download('all')
# Write your code here
!pip install preprocess
import preprocess
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Pac

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [24]:
import pyLDAvis
pyLDAvis.enable_notebook()

In [25]:
top_n = 10
topic_to_topwords = {}
for j, topic_to_word in enumerate(doxyDonkeyPosts):
    top = np.argsort(topic_to_word)[::-1][:top_n]
    msg = 'Topic %i '  % j
    top_words = [doxyDonkeyPosts[i].strip()[:35] for i in top]
    msg += ' '.join(top_words)
    print(msg)
    topic_to_topwords[j] = top_words

Topic 0 b"SoftBank's $100 Billion Tech Fund
Topic 1 b"SoftBank's $100 Billion Tech Fund
Topic 2 b"SoftBank's $100 Billion Tech Fund
Topic 3 b"SoftBank's $100 Billion Tech Fund
Topic 4 b"SoftBank's $100 Billion Tech Fund
Topic 5 b"SoftBank's $100 Billion Tech Fund
Topic 6 b"SoftBank's $100 Billion Tech Fund
Topic 7 b"SoftBank's $100 Billion Tech Fund
Topic 8 b"SoftBank's $100 Billion Tech Fund
Topic 9 b"SoftBank's $100 Billion Tech Fund
Topic 10 b"SoftBank's $100 Billion Tech Fund
Topic 11 b"SoftBank's $100 Billion Tech Fund
Topic 12 b"SoftBank's $100 Billion Tech Fund
Topic 13 b"SoftBank's $100 Billion Tech Fund
Topic 14 b"SoftBank's $100 Billion Tech Fund
Topic 15 b"SoftBank's $100 Billion Tech Fund
Topic 16 b"SoftBank's $100 Billion Tech Fund
Topic 17 b"SoftBank's $100 Billion Tech Fund
Topic 18 b"SoftBank's $100 Billion Tech Fund
Topic 19 b"SoftBank's $100 Billion Tech Fund
Topic 20 b"SoftBank's $100 Billion Tech Fund
Topic 21 b"SoftBank's $100 Billion Tech Fund
Topic 22 b"SoftBank'

In [39]:
import warnings
warnings.filterwarnings('ignore')
prepared_data = pyLDAvis.prepare(doxyDonkeyPosts, doxyDonkeyPosts, 
                                 dat['doc_lengths'] * 1.0, dat['vocab'], dat['term_frequency'] * 1.0, mds='tsne')

NameError: ignored

## (4) (10 points) Generate K topics by using BERTopic, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here: 

https://colab.research.google.com/drive/1FieRA9fLdkQEGDIMYl0I3MCjSUKVF8C-?usp=sharing

In [28]:
!pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 90 kB 3.4 MB/s 
     |████████████████████████████████| 85 kB 3.0 MB/s 
     |████████████████████████████████| 5.2 MB 37.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 88 kB 5.1 MB/s 
     |████████████████████████████████| 636 kB 31.2 MB/s 
     |████████████████████████████████| 5.5 MB 24.2 MB/s 
     |████████████████████████████████| 1.3 MB 57.8 MB/s 
     |████████████████████████████████| 163 kB 57.1 MB/s 
     |████████████████████████████████| 7.6 MB 22.4 MB/s 
     |████████████████████████████████| 1.1 MB 21.7 MB/s 
  Created wheel for hdbscan: filename=hdbscan-0.8.29-cp37-cp37m-linux_x86_64.whl size=2340708 sha256=265039ab1d9b86fcfec9e7a088a2b9862d23a543605f69bc4d5589e4795c17dc
  Stored in directory: /root/.cache

In [30]:
from bertopic import BERTopic

topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True)
topics, probs = topic_model.fit_transform(doxyDonkeyPosts)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

Batches:   0%|          | 0/88 [00:00<?, ?it/s]

2022-11-07 04:38:10,151 - BERTopic - Transformed documents to Embeddings
2022-11-07 04:38:38,656 - BERTopic - Reduced dimensionality
2022-11-07 04:38:39,726 - BERTopic - Clustered reduced embeddings


In [32]:
freq = topic_model.get_topic_info(); 
freq.head(5)

,Topic,Count,Name
0,-1,673,-1_the_to_and_of
1,0,140,0_flipkart_snapdeal_india_in
2,1,93,1_investors_venture_companies_private
3,2,84,2_alibaba_china_jdcom_percent
4,3,69,3_china_hackers_chinese_the


In [33]:
topic_model.get_topic(0)

[('flipkart', 0.018511945361278617),
 ('snapdeal', 0.016017514961895293),
 ('india', 0.01587853148191805),
 ('in', 0.012484868440462387),
 ('and', 0.012376584737928473),
 ('rs', 0.0122598991589313),
 ('ecommerce', 0.011457697494585549),
 ('crore', 0.011390642000151793),
 ('the', 0.01054511398330252),
 ('of', 0.010107807827077396)]

In [34]:
topic_model.visualize_topics()
topic_model.visualize_term_rank()

In [35]:
topic_model.visualize_topics()
topic_model.visualize_term_rank()

In [36]:
topic_model.update_topics(doxyDonkeyPosts, topics, n_gram_range=(1, 2))

## (5) (10 extra points) Compare the results generated by the four topic modeling algorithms, which one is better? You should explain the reasons in details.

Comparison of the algorithms are as follows :
 

1.   All three algorithms have similar documents at the top and bottom categories. 
2.   LSA generates  better results than LTA and BERT because it has the ability to decompose the original matrix by information retreival method to maintain key topics.
3. LDA works by assumption that documents are generate from a several topics which further output words through probability distribution. The downside of the LDA2Vec is that the context vectors resemble typical word vectors, making them less interpretable.
4. The lda2vec model goes one step beyond the paragraph vector approach by working with document-sized text fragments and decomposing the document vectors into two different components. In the same spirit as the LDA model, a document vector is decomposed into a document weight vector and a topic matrix.
